In [1]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.special import expit  # For tanh (logistic function)
import pandas as pd
import math

In [2]:
def design_matrix(model_name, x):
    if model_name == "POLY-4":
        X = np.column_stack([np.ones(len(x)), x, x**2, x**3, x**4])
    elif model_name == "CUBIC":
        X = np.column_stack([np.ones(len(x)), x, x**2, x**3])
    elif model_name == "PAR":
        X = np.column_stack([np.ones(len(x)), x, x**2])
    elif model_name == "LIN":
        X = np.column_stack([np.ones(len(x)), x])
    elif model_name == "LIN0":
        X = x.reshape(-1, 1)  # Only x as a feature (no intercept)
    else:
        raise NotImplementedError(model_name)
    return X

In [3]:
def fit_model(model_name, x, y, thetas_guess=None):
    """Fits the specified model using statsmodels (OLS)."""

    X = design_matrix(model_name, x)
    model = sm.OLS(y, X)
    results = model.fit()
    return results
    
thetas_dict = {
    "POLY-4": [0.5, 0.2, -0.1, 0.05, -0.01],  # Example thetas
    "CUBIC": [0.4, 0.3, -0.1, 0.02],
    "PAR": [0.6, 0.1, -0.05],
    "LIN": [0.5, 0.2],
    "LIN0": [0.3],  # theta1 is fixed to 0
    "EXP": [0.5, 2.0],  # theta1, theta2 for EXP
    "EXP*": [0.5, 2.0, 0.2],  # theta1, theta2, theta3 for EXP*
}

In [4]:
def ll_score(y_pred, y, sigma2):
    """Average log-likelihood"""
    pi = math.pi
    ln = math.log
    MSE = np.mean((y_pred - y)**2)
    return -(1/2)*(ln(2*pi*sigma2) + MSE/sigma2)

In [5]:
def score(y_pred, y):
    return np.mean((y_pred - y)**2) * 100000

In [6]:
def generate_data(x, noise_std=0.0):
    """Generates data based on the specified model."""
    y = 0.5 + 0.5 * np.tanh(x - 2)
    if y is not None:
        y += np.random.normal(0, noise_std, size=len(x)) # Add noise
    return y

def generate_x(x_range):
    step = 0.1
    return np.round(np.arange(x_range[0], x_range[1] + step, step), decimals=1)

# Set noise level (to zero, as specified in the prompt)
noise_std = 0.0  # No noise, as per the description

# Define x ranges (as specified in the text)
x_ranges = {
    "X0": (0, 3.5),
    "Xtarg": (3.6, 5),
    "Xall": (0, 5),  # Combined range for generalization
    "Xcal": (0, 2.5),
    "Xgen": (2.6, 3.5),
    "Xcal1": (0, 2.3),
    "Xgen1": (2.4, 3.5),
    "Xcal2": (0, 2.4),
    "Xgen2": (2.5, 3.5),
    "Xcal3": (0, 2.6),
    "Xgen3": (2.7, 3.5),
}

# Generate true y values
x_all = generate_x(x_ranges["Xall"])
y_true = generate_data(x_all, noise_std)  # TRUE model doesn't need thetas
f_dict = {x: y for x, y in zip(x_all, y_true)}
#plt.figure(figsize=(12, 8))
#plt.plot(x_all, y_true, label="TRUE", color="blue", linewidth=2)


In [ ]:
def run_simulation(x_range_names, models):
    """Runs the simulation and returns the results."""
    _x_ranges = {k: x_ranges[k] for k in x_range_names}
    results_dict = {}
    for model_name in models:
        results_dict[model_name] = {}
        for range_name, x_range in _x_ranges.items():
            # Generate x values, spaced by 0.1
            x = generate_x(x_range)

            # Generate data for the current model (with potential noise)
            thetas = thetas_dict.get(model_name)
            y = np.array([f_dict[x_i] for x_i in x])

            # Fit the model
            fit_results = fit_model(model_name, x, y, thetas)
            y_pred = fit_results.predict(fit_results.model.exog)

            results_dict[model_name][range_name] = {
                "score": score(y_pred, y),
                "fitted_params": fit_results.params if fit_results is not None else None,
                #"x": x,  # Store x and y for plotting
                #"y": y,
            }
    display(pd.DataFrame([[model] + [results_dict[model][range_name]["score"] for range_name in x_range_names] for model in models]))
    return results_dict

sim_models = list(reversed(["POLY-4", "CUBIC", "PAR", "LIN", "LIN0"]))
sim_range_names = ["Xcal", "X0", "Xall"]
results = run_simulation(sim_range_names, sim_models)


In [ ]:
def run_gen_simulation(range_names, models):
    """Runs the simulation and returns the results."""

    results_dict = {}
    for model_name in models:
        results_dict[model_name] = {}
        names = []
        for item in range_names: #range_name, x_range in _x_ranges.items():
            name = f"{item[0]}->{item[1]}"
            names.append(name)
            x_range = x_ranges[item[0]]
            # Generate x values, spaced by 0.1
            x = generate_x(x_range)

            # Generate data for the current model (with potential noise)
            thetas = thetas_dict.get(model_name)
            y = np.array([f_dict[x_i] for x_i in x])

            # Fit the model
            fit_results = fit_model(model_name, x, y, thetas)
            out_range = x_ranges[item[1]]
            x_out = generate_x(out_range)
            y_out = np.array([f_dict[x_i] for x_i in x_out])
            y_pred = fit_results.predict(design_matrix(model_name, x_out))

            results_dict[model_name][name] = {
                "score": round(score(y_pred, y_out), 0),
                "fitted_params": fit_results.params if fit_results is not None else None,
                "mse_resid": fit_results.mse_resid,
                "ll_score": ll_score(y_pred, y_out, fit_results.mse_resid)
                #"x": x,  # Store x and y for plotting
                #"y": y,
            }
    display(pd.DataFrame([[model] + [results_dict[model][range_name]["score"] for range_name in names] for model in models]))
    display(pd.DataFrame([[model] + [results_dict[model][range_name]["ll_score"] for range_name in names] for model in models]))
    return results_dict

sim_models = list(reversed(["POLY-4", "CUBIC", "PAR", "LIN", "LIN0"]))
sim_range_names = [("Xcal", "Xgen"), ("X0", "Xtarg"), ("X0", "Xall")]
results_gen = run_gen_simulation(sim_range_names, sim_models)
##### Table 2 #####


In [ ]:
temp0 = np.exp(np.array([[results_gen[model][range_name]["ll_score"] for range_name in results_gen[model]] for model in sim_models]))
temp0

In [ ]:
temp0[:, 0]/temp0[1, 0]

In [ ]:
step = 0.1
explore_range_names = []
for i in range(0, 10):
    cal_end = round(2.5 + step * i, 1)
    gen_start = round(cal_end + step, 1)
    cal_name = f"Xcal{cal_end:.1f}"
    gen_name = f"Xgen{cal_end:.1f}"
    x_ranges[cal_name] = (0, cal_end)
    x_ranges[gen_name] = (gen_start, 3.5)
    explore_range_names.append((cal_name, gen_name))
results_gen2 = run_gen_simulation(explore_range_names, sim_models)

In [ ]:
np.mean(np.array([[results_gen2[model][range_name]["score"] for range_name in results_gen2[model]] for model in sim_models]), axis=1)

In [ ]:
temp = np.exp(np.mean(np.array([[results_gen2[model][range_name]["ll_score"] for range_name in results_gen2[model]] for model in sim_models]), axis=1))
temp

In [ ]:
temp/temp[1]

In [ ]:
results = run_simulation(["Xcal2.6", "Xcal2.7", "Xcal2.8", "Xcal2.9", "Xcal3.0", "Xcal3.1", "Xcal3.2", "Xcal3.3", "Xcal3.4"], sim_models)

In [ ]:
x_ranges

In [ ]:
step = 0.1
explore_range_names = []
for i in range(0, 10):
    cal_end = round(2.5 + step * i, 1)
    gen_start = cal_end #round(cal_end + step, 1)
    cal_name = f"Xcal{cal_end:.1f}orig"
    gen_name = f"Xgen{cal_end:.1f}orig"
    x_ranges[cal_name] = (0, cal_end)
    x_ranges[gen_name] = (gen_start, 3.5)
    explore_range_names.append((cal_name, gen_name))
results_gen3 = run_gen_simulation(explore_range_names, sim_models)

In [ ]:
np.round(np.mean(np.array([[results_gen3[model][range_name]["score"] for range_name in results_gen3[model]] for model in sim_models]), axis=1), 0)

In [ ]:
model_name = "POLY-4"
x_range = x_ranges["X0"]

# Generate x values, spaced by 0.1
x = generate_x(x_range)

# Generate data for the current model (with potential noise)
thetas = thetas_dict.get(model_name)
y = np.array([f_dict[x_i] for x_i in x])

# Fit the model
fit_results = fit_model(model_name, x, y, thetas)
y_pred = fit_results.predict(fit_results.model.exog)
score(y_pred, y)

In [ ]:
#plt.figure(figsize=(12, 8))
x4 = generate_x(x_ranges["Xall"])
y4 = fit_results.predict(design_matrix("POLY-4", x4))
plt.plot(x_all, y_true, label="TRUE", color="blue", linewidth=2)
plt.plot(x4, y4, label="POLY-4", color="gray", linewidth=2)